# This notebook is the last step in preparing the data the Random Forest Regressors for Each population to Predict Biomass

 Here we will seperate the dataframes by population, prepare split features (variables used for prediction) and the label (what we are trying to predict, biomass).

This notebook also includes a few functions we will use within the notebook and within each population's random forest model building notebooks.

#### Set a working directory

In [29]:
# Set a working directory
#!pip install GitPython
import git
import os

repo = git.Repo('.', search_parent_directories=True)


os.chdir(repo.working_tree_dir)


In [30]:
import pandas as pd

covari_path = 'data_ingest/data/modified/RF_ready_covari.csv'
#using pandas to read in as a df
covari = (pd.read_csv(covari_path,parse_dates=[0]))
covari = covari.drop(columns = ['vgos', 'ugos'])
#taking a peak at the data
covari.head(3)



,time,cruise,lat,lon,biomass_pro,biomass_syn,biomass_pico,biomass_croco,ALK,sss,sst,Fe,O2,NO3,PO4,Si,hours_since_sunrise
0,2015-05-22 22:00:00,KM1508,21.343400,-158.273700,4.024661,0.337763,0.555395,0.009181,1952.6418,34.571716,25.653118,0.000088,216.794167,4.269278e-07,0.345151,9.464704,6.129444
1,2015-05-22 23:00:00,KM1508,21.343533,-158.273744,4.167834,0.413687,0.720884,0.013144,1952.6418,34.571716,25.653118,0.000088,216.794167,4.269278e-07,0.345151,9.464704,7.129444
2,2015-05-23 00:00:00,KM1508,21.346175,-158.274150,4.654360,0.654208,0.635654,0.008443,1952.6418,34.609317,25.646243,0.000088,216.794167,4.269278e-07,0.345151,9.464704,8.129722


### We have data for 4 types of phytoplankton, here we will split the df into one df per population

In [31]:
covari_pro = covari.drop(columns=['biomass_pico','biomass_croco','biomass_syn'])
covari_syn = covari.drop(columns=['biomass_pico','biomass_croco','biomass_pro'])
covari_pico = covari.drop(columns=['biomass_syn','biomass_croco','biomass_pro'])
covari_croco = covari.drop(columns=['biomass_pico','biomass_syn','biomass_pro'])

In [32]:
def population_dfer(covari_pop, pop_name):
    """
    This function removes population names from the columns of each df
    """
    df = covari_pop
    pop_name = pop_name
    df.rename(columns=lambda x: x.replace('_'+pop_name, ''), inplace=True)

In [33]:
#removing population names so columns are consistent accross dataframes
population_dfer(covari_syn, 'syn')
population_dfer(covari_pro, 'pro')
population_dfer(covari_pico, 'pico')
population_dfer(covari_croco, 'croco')


In [34]:
covari_syn = covari_syn.loc[covari_syn['cruise']!= 'RR2106']
covari_pico = covari_pico.loc[covari_pico['cruise']!= 'RR2106']
covari_syn = covari_syn.loc[covari_syn['cruise'] != 'RR1814']
covari_pico = covari_pico.loc[covari_pico['cruise'] != 'RR1814']

In [35]:
import numpy as np

def preprocess_single_population(covari_population, drop, cols_drop):
    """
    Takes the population dataframe and returns a dataframe that
    only includes the selected population's rows, a list of labels (biomass values associated with the dataframe)
    and a list of all of the features.
    """
    # Selecting the population based on the provided name
    pop_df = covari_population.copy()
    if drop:
        pop_df.drop(columns=cols_drop, inplace=True)

    # Creating the labels and features for the population
    labels = np.array(pop_df.biomass)
    labels = np.delete(labels, 0, 0)
    features = pop_df.drop(['time', 'biomass', 'lat', 'lon', 'cruise'], axis=1)
    # Saving feature names for later use
    feature_list = list(features.columns)
    features = features.to_numpy()
    features = np.delete(features, 0, 0)
    return pop_df, labels, features, feature_list




### Using the preprocess_single_population function for all of the populations

In [36]:
# creating a df, labels and features for the observed Prochlorooccus

drop = False
pro_df, labels_pro, features_pro, feature_list_pro = preprocess_single_population(covari_pro, drop, None)

In [37]:
# creating a df, labels and features for the observed Synechoccoccus


syn_df, labels_syn, features_syn, feature_list_syn = preprocess_single_population(covari_syn, drop, None)

In [38]:
# creating a df, labels and features for the observed Picoeukaryotes


pico_df, labels_pico, features_pico, feature_list_pico = preprocess_single_population(covari_pico, drop, None)

In [39]:
# creating a df, labels and features for the observed Nanoeukaryotes


croco_df, labels_croco, features_croco, feature_list_croco = preprocess_single_population(covari_croco, drop, None)

## Defining a function for finding the optimal testing to training ratio

Used in specific random forest model notebooks. This function graphs the Root Mean Square Error (RMSE) vs. the testing to training ratio for data used in the model.population's model.

In [40]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np

def testing_training_ratio(features, labels, feature_list, title_prefix):
    """
    This function uses K-fold cross validation to split the training and test sets, each population specific notebook will use this function
    """
    # Graphs the RMSE of different testing and training ratios
    RMSEs = {'Test_Ratio':[], 'RMSE': []}
    #define number of folds to try
    splits = [2,4,6,8,12,16]
    # Loop through the number of splits
    for n in splits:
        n_splits = n
        kf = KFold(n_splits=n_splits, shuffle=False)
        #fitting the models, making predictions and calculating RMSE
        for train_index, test_index in kf.split(features):
            train_features, test_features = features[train_index], features[test_index]
            train_labels, test_labels = labels[train_index], labels[test_index]

            rf = RandomForestRegressor(n_estimators = 80, max_depth= 20, max_features='sqrt', random_state = 42)
            rf.fit(train_features, train_labels)

            # Use the forest's predict method on the test data
            predictions = rf.predict(test_features)

            # Calculate the absolute errors
            errors = abs(predictions - test_labels)

            # Finding the root mean square error (RMSE)
            RMSE = mean_squared_error(test_labels, predictions, squared=False) #setting squared=False gives us RMSE not MSE
            RMSEs['RMSE'].append(RMSE)
            RMSEs['Test_Ratio'].append(1/n_splits)  # The test ratio for n-fold cross-validation is 1/n_splits
   
    test_ratios = RMSEs['Test_Ratio']
    rmse_values = RMSEs['RMSE']

    # Create a line plot
    plt.figure(figsize=(10, 6))  
    plt.plot(test_ratios, rmse_values, marker='o')

   
    plt.fill_between(test_ratios, rmse_values, alpha=0.3)
    
    plt.xlabel('Testing:Training Ratio', fontsize=15)
    plt.ylabel('RMSE of Biomass (pgC/L)fn', fontsize=15)
    plt.title(f"{title_prefix} - RMSE of Biomass vs. Testing: Training Ratio", fontsize=22)

    plt.xlim(0, 1)  
    
    plt.xticks([i/10 for i in range(11)])  # Set the x-axis tick locations at 0.1 increments
    plt.gca().invert_xaxis()

    
    plt.grid(True) 
    
    plt.tight_layout()  # Improves spacing between the plot elements
    plt.savefig(f"figures/{title_prefix}/RMSEsByFolds.png")
    plt.show()
    
    

    return RMSEs



In [41]:
def testing_training_ratio_random(features, labels, feature_list, title_prefix):
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import mean_squared_error
    import matplotlib.pyplot as plt
    """
    Used in specific random forest model notebooks. This function graphs the Root Mean Square Error (RMSE) vs.
    the ratio to testing to training data. 
    """
    # Graphs the RMSE of differnt testing and training ratios
    RMSEs = {'Test_Ratio':[], 'RMSE': []}
    
    range_list = [i / 20.0 for i in range(1, 19)] + [0.9][:-1] #prints 0.9 twice so I use all but last value

    for fifth in range_list:
        fifth = round(fifth, ndigits=2)
        RMSEs['Test_Ratio'].append(fifth)
        #using train_test_split to manipulate the training to testing ratio
        train_features, test_features, train_labels, test_labels = train_test_split(
            features, labels, test_size = fifth, random_state = 42
            
        )
        rf = RandomForestRegressor(n_estimators = 80, max_depth = 10, max_features='sqrt', random_state = 42)
        rf.fit(train_features, train_labels)
        
        
        # Convert test_features to a DataFrame
        test_features_df = pd.DataFrame(test_features, columns=feature_list)

        # Use the forest's predict method on the test data
        predictions = rf.predict(test_features)

        # Create a new Series with predicted values and index from test_features_df
        predic_biomass = pd.Series(predictions, index=test_features_df.index)

        # Assign the new Series to the DataFrame using .loc
        test_features_df.loc[:, 'Prediction'] = predic_biomass

        # Calculate the absolute errors
        errors = abs(predictions - test_labels)

        # Finding the root mean square error (RMSE)

        # RMSE give realtively high weight to large errors 
        RMSE = mean_squared_error(test_labels, predictions, squared=False) #setting squared=False gives us RMSE not MSE
        RMSEs['RMSE'].append(RMSE)
        
    
    # Extract Test Ratios and RMSEs from the dictionary
    test_ratios = RMSEs['Test_Ratio']
    rmse_values = RMSEs['RMSE']

    # Create a line plot
    plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
    plt.plot(test_ratios, rmse_values, marker='o')

    # Fill the area under the curve
    plt.fill_between(test_ratios, rmse_values, alpha=0.3)
    
    plt.xlabel('Testing:Training Ratio', fontsize=15)
    plt.ylabel('RMSE of Biomass (pgC/L)fn', fontsize=15)
    plt.title(f"{title_prefix} - RMSE of Biomass vs. Testing: Training Ratio", fontsize=22)

    plt.xlim(0, 1)  # Set the x-axis limits from 0 to 1
    
    plt.xticks([i/10 for i in range(11)])  # Set the x-axis tick locations at 0.1 increments
    #inversing the x axis
    plt.gca().invert_xaxis()

    
    plt.grid(True)  # Add a grid to the plot
    
    plt.tight_layout()  # Improves spacing between the plot elements
    plt.show()
    
    return RMSEs

##  Functions for cross validation

In [42]:
def val_set(features, labels):
    '''
    This function uses a validation set to test the model
    this uses the first 600 data points for testing and the rest for training
    '''
    test_features = features[:600]
    train_features = features[600:]
    test_labels = labels[:600]
    train_labels = labels[600:]
    rf = RandomForestRegressor(n_estimators = 80, max_depth = 10, max_features='sqrt', random_state = 42)
    rf.fit(train_features, train_labels)
    predictions = rf.predict(test_features)
    RMSE = mean_squared_error(test_labels, predictions, squared=False)

    plt.figure(figsize=(10, 6))
    plt.plot(test_labels, label='True Biomass')
    plt.plot(predictions, label='Predicted Biomass')
    plt.xlabel('Observation')
    plt.ylabel('Biomass (pgC/L)')
    plt.title('Validation set - True vs. Predicted Biomass')
    plt.legend()
    plt.show()
    return RMSE

In [43]:
fold = [0,1,2,3,4,5,6,7]

In [44]:
def equalize(data):
    '''
    this function makes all of our folds the same length for k-fold cross validation
    '''
    # Find the length of the shortest sublist
    min_len = min(len(sublist) for sublist in data)

    # Remove data points from each sublist until they are all the same length
    data_truncated = [sublist[:min_len] for sublist in data]

    return data_truncated

In [45]:
from sklearn.model_selection import KFold
import numpy as np

def k_fold(features, labels, splits):
    # initialize kfold
    n_splits = splits
    kf = KFold(n_splits=n_splits, shuffle=False)

    # Initialize lists to hold training and testing data
    train_features = []
    test_features = []
    train_labels = []
    test_labels = []

    # Split the data into training and testing sets for each fold
    for train_index, test_index in kf.split(features):
        train_feat, test_feat = features[train_index], features[test_index]
        train_lab, test_lab = labels[train_index], labels[test_index]
        
        # Append the training and testing data for this fold to the lists
        train_features.append(train_feat)
        test_features.append(test_feat)
        train_labels.append(train_lab)
        test_labels.append(test_lab)
    return train_features, test_features, train_labels, test_labels

In [46]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import joblib
def model_training(train_features, train_labels, test_features, test_labels, hyperparameters, pop_name):
    '''
    this function trains a random forest regressor model for each fold and saves the model
    '''
    # make sure features and labels are the same length
    train_features = equalize(train_features)
    test_features = equalize(test_features)
    train_labels = equalize(train_labels)
    test_labels = equalize(test_labels)


    train_features = np.array(train_features)
    train_labels = np.array(train_labels)
    test_features = np.array(test_features)
    test_labels = np.array(test_labels)

    
    models = []
    n_estimators = hyperparameters['n_estimators']
    max_depth = hyperparameters['max_depth']
    max_features = hyperparameters['max_features']
    if 'criterion' in hyperparameters.keys():
        criterion = hyperparameters['criterion']
    else:
        criterion = 'squared_error'
    # fit the models
    for i in range(train_features.shape[0]):
        
        rf = RandomForestRegressor(n_estimators = n_estimators, max_features=max_features, max_depth = max_depth, criterion = criterion, random_state = 44)
        
    
        rf.fit(train_features[i], train_labels[i])
        
        
        models.append(rf)

    # Save the models
    for i, model in enumerate(models):
        joblib.dump(model, f"RF_models/{pop_name}/random_forest_fold_{i}.joblib")

In [47]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib
import pandas as pd

def predict_kfold(test_features, test_labels, train_labels, train_features, pop_name):
    """
    This function uses the models trained in the model_training function to predict biomass values for each fold
    """
    
    predictions = []
    maes = []
    rmses = []
    

    train_features = equalize(train_features)
    test_features = equalize(test_features)
    train_labels = equalize(train_labels)
    test_labels = equalize(test_labels)

    train_features = np.array(train_features)
    train_labels = np.array(train_labels)
    test_features = np.array(test_features)
    test_labels = np.array(test_labels)
    
    for i in range(test_features.shape[0]):
        # Load the model for this fold
        rf = joblib.load(f"RF_models/{pop_name}/random_forest_fold_{i}.joblib")
        
        # Use the model to predict on the test data for this fold
        preds = rf.predict(test_features[i])
        
        # Calculate the errors
        mae = mean_absolute_error(test_labels[i], preds)
        RMSE = mean_squared_error(test_labels[i], preds, squared=False)

        # Append the predictions and errors to the lists
        predictions.append(preds)
        maes.append(mae)
        rmses.append(RMSE)
        # Save the predictions for each fold
        data = {'predictions': preds,
            'reals' : test_labels[i]}
        
        # for key, value in data.items():
        #     print(f"Number of elements in {key}: {np.size(value)}")
        
        actual = pd.DataFrame(data)
        actual.to_csv(f'data_ingest/data/modified/actual_{pop_name}{i}.csv', index=False)


    # Convert lists of arrays to 2D arrays

    predictions = np.concatenate(predictions)
    maes = np.array(maes)
    rmse = np.mean(rmses)
    rmses = np.array(rmses)
    #rmse = np.sqrt(np.mean(rmses**2))

    # Print the mean absolute errors and root mean square errors
    return predictions, maes, rmses, rmse

In [48]:

import pandas as pd
from plotly.subplots import make_subplots
import seaborn as sns
#suppress future warnings (since we are version controlled and don't need to see them)
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

def plot_predictions(pop_name):
    """
    This function plots the predicted vs. actual biomass values for each fold
    """

    plt.figure(figsize=(8,14))


    for i, f in enumerate(fold):
        actual = pd.read_csv(f'data_ingest/data/modified/actual_{pop_name}{i}.csv')

        plt.subplot(4, 2, i+1)  # Create a subplot for each fold

        sns.kdeplot(x='reals', y='predictions', data=actual, fill=True, cmap="Reds", thresh=0.05,)
        plt.plot([0, 1], [0, 1], transform=plt.gca().transAxes, ls='--', c='black')
        plt.xlim(0, 20)
        plt.ylim(0, 20)
        if pop_name == 'syn':
            plt.xlim(0, 6)
            plt.ylim(0, 6)
        if pop_name == 'pico':
            plt.xlim(0, 15)
            plt.ylim(0, 15)
        
        plt.title(f'{pop_name} Fold {f+1}')
        

    plt.tight_layout()  # Adjust the layout so that the plots do not overlap
    plt.show()



    # for f in fold:
    #     actual = pd.read_csv(f'data_ingest/data/modified/actual_pro{f}.csv')
    #     # #Create scatter reals v preds
    #     #sns.scatterplot(x='reals', y='predictions', data=actual)
    #     #create density plot
    #     sns.kdeplot(x='reals', y='predictions', data=actual, fill=True, cmap="Reds", thresh=0.05,)
    #     # Add a reference line from (0,0) to (1,1)
    #     plt.plot([0, 1], [0, 1], transform=plt.gca().transAxes, ls='--', c='black')
    #     plt.xlim(0, 20)
    #     plt.ylim(0, 20)

    #     # #linear regression line
    #     # x = data['reals']
    #     # y = data['predictions']
    #     # slope, intercept = np.polyfit(x, y, 1)
    #     # plt.plot(x, slope*x + intercept, color='blue', label='Linear Regression')
        
    #     plt.title(f'Prochlorococcus Fold {f+1}')
    #     plt.savefig(f'figures/pro_fold{f+1}.png')
    #     plt.show()

In [49]:
def covari_predictions(predictions, pop_name):
    """
    This function merges the predictions with the original covari data
    """
    if pop_name == 'pro':
        original_df = covari_pro.reset_index()

    elif pop_name == 'syn':
        original_df = covari_syn.reset_index()
    
    elif pop_name == 'pico':
        original_df = covari_pico.reset_index()

    elif pop_name == 'croco':
        original_df = covari_croco.reset_index()

    
    predicted_df = pd.DataFrame(predictions, columns=['prediction'])
    #adding header to predictions
    head = 'predicted'
    header = pd.DataFrame([head], index=[0])
    # Concatenate the new row and the existing DataFrame
    predicted_df = pd.concat([header, predicted_df]).reset_index()

    length = len(predicted_df)
    original_df = original_df[:length]
    predicted_df['index'] = original_df.index




    # Merge the two dataframes on the index
    merged_df = pd.merge(original_df, predicted_df, on='index')
    # Set the index back to the original column
    merged_df = merged_df.set_index('index')
    merged_df.head(5)
    return merged_df


In [50]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def create_fold_predictions_time(pop_df, pop_name):
    """
    Makes an actual vs prediction plot for each cruise, so we can compare real points
    to prediction points
    """
    unique_cruises = pop_df['cruise'].unique()

    # Create a subplot grid
    num_cruises = len(unique_cruises)
    rows = int(num_cruises / 2) if num_cruises % 2 == 0 else int(num_cruises / 2) + 1
    fig = make_subplots(rows=rows, cols=2, subplot_titles=unique_cruises)

    
    prediction_color = 'red'

    colors = ['blue', 'green'] 

    # we will use this point counter to measure which data is from which fold
    point_counter = 0  

    y_range = [0, 100]
    if pop_name == 'syn':
        y_range = [0, 12]
    if pop_name == 'pico':
        y_range = [0, 30]
    


    for i, cruise in enumerate(unique_cruises):
        cruise_df = pop_df[pop_df['cruise'] == cruise]

        cruise_color_list = [colors[(point_counter + j) // 584 % len(colors)] for j in range(len(cruise_df))]

        point_counter += len(cruise_df)

        row = int(i / 2) + 1
        col = i % 2 + 1
        fig.add_trace(go.Scatter(x=cruise_df['time'], y=cruise_df['biomass'], mode='markers', name='Actual',
                                marker=dict(color=cruise_color_list)),
                    row=row, col=col)
        fig.add_trace(go.Scatter(x=cruise_df['time'], y=cruise_df['prediction'], mode='lines', name='Prediction',
                                line=dict(color=prediction_color)),
                    row=row, col=col)
        fig.update_xaxes(title_text='Time', row=row, col=col)
        fig.update_yaxes(title_text='Biomass (pCg/L)', row=row, col=col)
        fig.update_yaxes(range=y_range, row=row, col=col)

    fig.update_layout(height=600 * rows, width=1200, title_text='Actual and Prediction for Each Cruise')
    fig.show()



In [51]:
def RMSE_fold(folds, return_rmse):
    train_features, test_features, train_labels, test_labels = k_fold(features_pro, labels_pro, folds)
    model_training(train_features, train_labels, test_features, test_labels, hyperparameters={'n_estimators': 200, 'max_depth': 10, 'max_features': 'sqrt'})
    predictions, maes, rmses, rmse = predict_kfold(test_features, test_labels, train_labels, train_features)
    
    print("The RMSE for " + str(folds) + " fold cross validation is:" + str(rmse))
    if return_rmse:
        return rmse
    

#### Defining a function to plot out-of-bag error (OOB) againts number of trees in random forest model
currently unused

In [52]:
from collections import OrderedDict
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

def plot_oob_error_vs_num_trees(train_features, train_labels, title_prefix):
    """
    Developes a plot of Out of Bag (oob) error vs the number of trees grown in a random forest model. There are
        three labeled lines within the plot one representing.
    
    Max features represent the amount of all features (varaibles we are predicting on) used for each 
        tree in the random forest. n = all features.
    
    Warm start = true:reuse the solution of the previous call to fit and add more
        estimators to the ensemble, otherwise, just fit a whole new forest.
        
    oob_score = True: Use out-of-bag samples to estimate the generalization score. By default, r2_score is used.
        Provide a callable with signature.
    
    random state: controls random number generator that is used to shuffle/split the data. Ensures the same
        randomization is used each time the code is ran.
    
    """
    RANDOM_STATE = 42

    ensemble_clfs = [
        (
            "max_features='sqrt(n)'",
            RandomForestRegressor(
                warm_start=True,
                max_features="sqrt",
                oob_score=True,
                random_state=RANDOM_STATE,
            ),
        ),
        (
            "max_features='1/3 n'",
            RandomForestRegressor(
                warm_start=True,
                max_features=1/3,
                oob_score=True,
                random_state=RANDOM_STATE,
            ),
        ),
        (
            "max_features= n",
            RandomForestRegressor(
                warm_start=True,
                max_features=None,
                oob_score=True,
                random_state=RANDOM_STATE,
            ),
        ),
    ]

    error_rate = OrderedDict((label, []) for label, _ in ensemble_clfs)

    min_estimators = 15
    max_estimators = 128

    for label, clf in ensemble_clfs:
        for i in range(min_estimators, max_estimators + 1, 5):
            oob_errors = []
            for fold_features, fold_labels in zip(train_features, train_labels):
                clf.set_params(n_estimators=i)
                clf.fit(fold_features, fold_labels)
                oob_error = 1 - clf.oob_score_
                oob_errors.append(oob_error)
            avg_oob_error = np.mean(oob_errors)
            error_rate[label].append((i, avg_oob_error))
    for label, clf_err in error_rate.items():
        xs, ys = zip(*clf_err)
        plt.plot(xs, ys, label=label)

    plt.xlim(min_estimators, max_estimators)
    plt.xlabel("# of Trees")
    plt.ylabel("OOB error rate (1 - R^2)")
    plt.legend(loc="upper right")
    plt.suptitle(f"{title_prefix} - Out-of-Bag Error Rate vs. Number of Trees in Random Forest Regression")
    plt.show()


## Functions for feature and hyperparameter tuning

In [53]:
def predict_cruise(hyperparameters, pop_name):
    """
    this function retrains the model on new hyperparameters and predicts ona single cruise (KM2010)
    """
    model_training(train_features, train_labels, test_features, test_labels, hyperparameters, pop_name)
    predictions, maes, rmses, rmse = predict_kfold(test_features, test_labels, train_labels, train_features, pop_name)
    merged_df = covari_predictions(predictions, pop_name)
    
    # plot the prediction for KM2010
    fig = go.Figure()
    cruise_df = merged_df[merged_df['cruise'] == 'KM2010']
    fig.add_trace(go.Scatter(x=cruise_df['time'], y=cruise_df['biomass'], mode='markers', name='Actual' , marker=dict(color='blue')))
    fig.add_trace(go.Scatter(x=cruise_df['time'], y=cruise_df['prediction'], mode='lines', name='Prediction',
                                line=dict(color='red')))
    fig.update_layout(title='KM2010', xaxis_title='Time', yaxis_title='Biomass')
    fig.show()

In [54]:
import matplotlib.pyplot as plt
%matplotlib inline

def feature_importance(pop_name, feature_list):
    feature_importance = []
    for f in fold:
        rf = joblib.load(f"RF_models/{pop_name}/random_forest_fold_{f}.joblib")
        feat_importance = pd.DataFrame(rf.feature_importances_, index=feature_list).sort_values(by=0, ascending=False)
        feature_importance.append(feat_importance)
    feature_importance = pd.concat(feature_importance, axis=1, keys=fold,)
    feature_importance.columns = feature_importance.columns.droplevel(1)
    feature_importance['mean'] = feature_importance.mean(axis=1)
    feature_importance = feature_importance.sort_values(by='mean', ascending=False)

    plt.style.use('fivethirtyeight')
    # Make a bar chart
    plt.bar(x=feature_importance.index, height=feature_importance['mean'], orientation = 'vertical')
    # Tick labels for x axis
    plt.xticks(feature_importance.index, rotation=45, ha='right', rotation_mode='anchor')

# # Axis labels and title
# plt.ylabel('Importance'); plt.title('Variable Importances for Pro RF')

    # fig, axs = plt.subplots(4, 2, figsize=(10, 20))
    # axs = axs.flatten()


    # for i, f in enumerate(fold):
    #     # Sort the data in descending order and plot
    #     feature_importance[f].sort_values(ascending=False).plot(kind='bar', ax=axs[i])
    #     axs[i].set_title(f'Fold {f+1}')


    plt.tight_layout()
    plt.show()

In [55]:
from sklearn.inspection import permutation_importance
fold = [0,1,2,3,4,5,6,7]
def permutation_importances(population, test_features1, test_labels1, feature_list):
    feature_importances = pd.DataFrame(index = feature_list, columns = fold)
    feature_deviation = pd.DataFrame(index = feature_list, columns = fold)
    for f in fold:
        test_features_df = pd.DataFrame(test_features1[f])
        test_labels_df = pd.DataFrame(test_labels1[f])
        rf = joblib.load(f"RF_models/{population}/random_forest_fold_{f}.joblib")
        result = permutation_importance(
            rf, test_features_df, test_labels_df, n_repeats=10, random_state=33, scoring = 'neg_root_mean_squared_error', n_jobs=2)
        # print(result.importances_mean)
        feature_importances[f] = result.importances_mean
        feature_deviation[f] = result.importances_std


    forest_importances = feature_importances.mean(axis=1)
    deviation = feature_deviation.mean(axis=1)
    forest_importances = forest_importances.sort_values(ascending=False)
    deviation = deviation[forest_importances.index]
    fig, ax = plt.subplots()
    forest_importances.plot.bar(yerr=deviation, ax=ax)
    ax.set_title("Feature importances using permutation")
    ax.set_ylabel("Mean accuracy decrease")
    
    plt.show()


In [ ]:
def permutation_full(features, labels, feature_list):
    rf = RandomForestRegressor(n_estimators = 120, max_features='sqrt', max_depth = 12, random_state = 44)
        
    
    rf.fit(features, labels)

    result = permutation_importance(
        rf, features, labels, n_repeats=10, random_state=33, scoring = 'neg_root_mean_squared_error', n_jobs=2)
    forest_importances = result.importances_mean
    forest_importances = pd.Series(forest_importances, index=feature_list)
    forest_importances = forest_importances.sort_values(ascending=False)
    fig, ax = plt.subplots()
    forest_importances.plot.bar(yerr=result.importances_std, ax=ax)
    ax.set_title("Feature importances using permutation")
    ax.set_ylabel("Mean accuracy decrease")

    plt.show()

In [56]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

def grid_search_hyperparams(param_grid, metrics, features, labels):
    """
    Uses a function from sklearn to test different combinations of hyperparameters and find the ones 
    with the lowest value of the specified test metric.  Do not rely on these hyperparameters as the best
    for prediction, as we need the model to be more flexible than is represented by the test data.
    """
    
    rf = RandomForestRegressor(random_state=42)

    numK=[8]

    for i in numK:
        for metric in metrics:
            grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=i, scoring=metric, error_score='raise', verbose=1)

            grid_search.fit(features, labels)

            # Get the best hyperparameters
            best_params = grid_search.best_params_

            print("Best hyperparameters for ", metric, f"are: {best_params}")